# PROBLEM DEFINITION
The production system consists of six machines, each considered unbreakable (no maintenance required).

This production system is responsible for producing three different families of products, each family needs a different floor routing and different processing tims for each machine

## DATA
- **6 machines**
- **3 product families**
- **Job arrival rate (exponential)** lambda 0.65
- **Families’ weights**
    - F1: 10%
    - F2: 52%
    - F3: 38%
- **Families' floor routings**

    ![Testo alternativo](floor_routing.png)
- **Families processing times**
    - F1: gamma distribution, alpha=2, beta=2
    - F2: gamma distribution, alpha=4, beta=0.5
    - F2: gamma distribution, alpha=6, beta=1/6
- **Due dates**
Uniform: U(30, 50)


## GOAL 1

Students must implement a simulation of the environment managed using the “PUSH” policy: whenever a new customer order is received, it is immediately released into the shop ﬂoor.

The PUSH policy serves as the benchmark for comparison with the RL-based solution.

# GOAL 1, structure

In [249]:
import simpy
from typing import Dict, Tuple

In [250]:
class Family:
    def __init__(self, name, floor_routing, processing_time, arrival_rate):
        self.name : str = name
        self.floor_routing : Dict[str, int] = floor_routing
        self.processing_time: Tuple[int, int] = processing_time
        self.arrival_rate : int = arrival_rate
    
    def machine_processing_time(self, job_rng):
        return job_rng.gammavariate(alpha = self.processing_time[0], beta = self.processing_time[1])
    
    def __str__(self):
        return self.name
    
FAMILIES = {"F1" : Family("F1", {"WC1": 1, "WC2": 1, "WC3": 0, "WC4": 1, "WC5":1, "WC6": 1}, (2, 2), 10),
            "F2" : Family("F2", {"WC1": 0.8, "WC2": 0.8, "WC3": 1, "WC4": 0.8, "WC5":0.8, "WC6": 0.75}, (4, 0.5), 52),
            "F3" : Family("F3", {"WC1": 0, "WC2": 0, "WC3": 1, "WC4": 0, "WC5":0, "WC6": 0.75}, (6, 1/6), 38)
            }  

In [251]:
from colorama import Fore, Style, init
init(autoreset=True)

FAMILIES_OUTPUT_COLORS = {
    "F1" : Fore.LIGHTRED_EX,
    "F2" : Fore.LIGHTYELLOW_EX,
    "F3" : Fore.LIGHTGREEN_EX    
}

In [252]:
from IPython.display import Markdown, display

FAMILIES_OUTPUT_COLORS = {
    "F1": "red",
    "F2": "orange",
    "F3": "green"
}
def color_print(family_name, text):
    color = FAMILIES_OUTPUT_COLORS[family_name]
    display(Markdown(f"<span style='color:{color}; font-weight:bold'>{text}</span>"))


Generate task of random family

In [253]:
import random

FAMILY_SEED = 45
family_rng = random.Random(FAMILY_SEED)
def sample_family() -> Family:
    family_names = list(FAMILIES.keys())
    family_probability = list(FAMILIES[name].arrival_rate for name in family_names)
    
    sampled_family = family_rng.choices(family_names, weights=family_probability, k = 1)[0]
    return Family(sampled_family, FAMILIES[sampled_family].floor_routing, FAMILIES[sampled_family].processing_time, FAMILIES[sampled_family].arrival_rate )

Shape each machine as a simpy.Resource with capacity 1.
Not one simpy.Resource with capacity = 6 because the machines differ one from another.

In [254]:
class Machine(simpy.Resource):
    def __init__(self, type_name : str, env: simpy.Environment, capacity: int = 1) -> None:
        super().__init__(env, capacity)
        self.env = env
        self.type_name = type_name

Shape each job, making it flow in machines according to its family

In [255]:
class Job:
    def __init__(self, env, family, id, this_job_rng):
        self.env = env
        self.family = family
        self.this_job_rng = this_job_rng
        self.id = id
        
    def process(self, shopfloor):
        for machine_name, machine_probability in self.family.floor_routing.items():
            if self.this_job_rng.random() < machine_probability:
                with shopfloor.machines[machine_name].request() as request:
                    #start waiting to be processed
                    yield request
                    #end of waiting
                    yield self.env.timeout(self.family.machine_processing_time(self.this_job_rng))
            else:
                self.this_job_rng.random()
                #to consume random numbers in same order
        
        color_print(self.family.name,f"Order just delivered! id:{self.id}")
                    

## RANDOM NUMBERS REPRODUCIBILITY
one seed for:
1. JOB_SEED -->
Jobs's creation rate + job's seed
2. THIS_JOB_SEED-->
Inside each job, to determine if a the job pass through a machine + waiting time in that machine
If that machine is not visited, random number is generated anyway
3. FAMILY_SEED --> 
Family's creation rate

Python implementation:
```python
this_job_rng_seed = job_rng.randint(0, 10_000_000)
this_job_rng = random.Random(this_job_rng_seed)
```
This because ```random.Random()``` needs an integer parameter while ```job_rng.random() generates a float in between [0,1)
            

Shape shop floor:
* generates jobs
* decide when to push jobs in the sho floor (with PUSH) policy
* measure statistics

In [256]:
JOB_SEED = 67
job_rng = random.Random(JOB_SEED)

class ShopFloor:
    def __init__(self, env: simpy.Environment):
        self.env = env
        self.orders = []
        self.job_id = 1
        
        self.machines = {
            "WC1": Machine("WC1", env),
            "WC2": Machine("WC2", env),
            "WC3": Machine("WC3", env),
            "WC4": Machine("WC4", env),
            "WC5": Machine("WC5", env),
            "WC6": Machine("WC6", env)         
        }
    
    def job_generator(self):
        while True:
            yield self.env.timeout(job_rng.expovariate(lambd= 0.65))
            
            this_job_rng_seed = job_rng.randint(0, 10_000_000)
            this_job_rng = random.Random(this_job_rng_seed)
            
            new_job = Job(self.env, sample_family(), self.job_id, this_job_rng)
            self.job_id += 1
            self.orders.append(new_job)
            
            color_print(new_job.family.name, f'New order! It is {new_job.family} id:{new_job.id}')

    def run(self):
        while True:
            if self.orders:
                job = self.orders.pop(0)
                self.env.process(job.process(self))
            yield self.env.timeout(0.1)  #to be able to simulate

Run simulation

In [257]:
env = simpy.Environment()
shopfloor = ShopFloor(env = env)

env.process(shopfloor.job_generator())
env.process(shopfloor.run())

env.run(until = 60)

<span style='color:orange; font-weight:bold'>New order! It is F2 id:1</span>

<span style='color:orange; font-weight:bold'>New order! It is F2 id:2</span>

<span style='color:red; font-weight:bold'>New order! It is F1 id:3</span>

<span style='color:orange; font-weight:bold'>New order! It is F2 id:4</span>

<span style='color:red; font-weight:bold'>New order! It is F1 id:5</span>

...

<span style='color:orange; font-weight:bold'>Order just delivered! id:7</span>

<span style='color:green; font-weight:bold'>New order! It is F3 id:36</span>

<span style='color:green; font-weight:bold'>New order! It is F3 id:37</span>

<span style='color:green; font-weight:bold'>Order just delivered! id:29</span>

<span style='color:green; font-weight:bold'>New order! It is F3 id:38</span>

## GOAL 2

Students must implement an alternative environment where, upon receiving a customer order, the order is placed into a “pre-shop pool” (PSP). At regular intervals, a reinforcement learning (RL) agent will decide whether to release the most urgent order from the PSP into the shopﬂoor.
The objective is to achieve:
- the same throughput as the PUSH system
- a comparable job tardiness 
- a comparable job earliness 
- lower WIP